In [58]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF
from SToC import *

import numpy as np
import os

from IPython.display import HTML, display
from tabulate import tabulate
from metrics import Evaluation as Eval

In [6]:
def print_join(iterates, model, W, n_topics, features, n_words):
	more = []
	tops = []

	more2 = []

	for i in range(len(iterates)):
		a = iterates[i][1]
		b = iterates[i][2]

		#mudar

		if ((a<n_topics)&(b>=n_topics)):
			aux = (model[a] + more[b-n_topics])/2
			#aux = more[b-n_topics]
			aux2 = (W[a] + more2[b-n_topics])/2
#
		if ((a>=n_topics)&(b<n_topics)):
			aux = (more[a-n_topics] + model[b])/2
			#aux = model[b]
			aux2 = (more2[a-n_topics] + W[b])/2

		if ((a>=n_topics)&(b>=n_topics)):
			#print a
			#print b
			#print len(more)
			aux = (more[a-n_topics] + more[b-n_topics])/2
			#aux = more[b-n_topics]# + more[b-n_topics])/2
			aux2 = (more2[a-n_topics] + more2[b-n_topics])/2
		
		if ((a < n_topics) & (b < n_topics)):
			aux = (model[a] + model[b])/2
			#aux = model[b]
			aux2 = (W[a] + W[b])/2

			
		#print aux
		more.append(aux)
		more2.append(aux2)
def print_estats(t2):

	valores = []
	for (i,j,k,l) in t2:
		valores.append(l)

	ex =  pd.Series(valores)
	print ("(Max+min)/2: ", (ex.max() + ex.min())/2)
	print ("Media: ", ex.mean())
	print( "Max: ", ex.max())
	print ("Min: ", ex.min())
	print ("Variancia: ", ex.var())
	print ("Desvio Padrao: ", ex.std())
	print ("Desvio Absoluto: ", ex.mad())
	return ex.mean()

def see_join(iterates, W, H, limiar,n_final):

    W = W.transpose()
    H = H.transpose()

    W_dict = {}
    H_dict = {}

    for i in range(len(W)):
        W_dict[i] = W[i]
    
    for i in range(len(H)):
        H_dict[i] = H[i]

	#iterates[i][0] numero do novo topicos
	#iterates[i][1] uniao tópico 1
	#iterates[i][2] uniao topico 2
	#iterates[i][3] valor (limiar)

 

	#iterates = sorted(iterates, key=lambda x: x[3], reverse=True)

	#print iterates
	#exit()
    n_topics = len(W)
    aux_final = len(W)
    print(aux_final)
    for idx,i in enumerate(range(len(iterates))):
        new = iterates[i][0]
        a = iterates[i][1]
        b = iterates[i][2]
        valor = iterates[i][3]
        
        #if valor < limiar:
        #	break

        if aux_final == n_final:
            break

        W_dict[new] = (W_dict[a] + W_dict[b])/2
        del(W_dict[a])
        del(W_dict[b])

        H_dict[new] = (H_dict[a] + H_dict[b])/2
        del(H_dict[a])
        del(H_dict[b])



        aux_final -=1

    W_new = []

    H_new = []

        #print (W_dict.keys())
        #exit()

        #print sorted(W_dict.keys())

    for i in sorted(W_dict.keys()):
        W_new.append(W_dict[i])

    for i in sorted(H_dict.keys()):
        H_new.append(H_dict[i])

    W_ret = np.zeros((len(W_new),len(W_new[0])))
    H_ret = np.zeros((len(H_new),len(H_new[0])))

    for i in range(len(W_new)):
        for j in range(len(W_new[0])):
            W_ret[i][j] = W_new[i][j]

    for i in range(len(H_new)):
        for j in range(len(H_new[0])):
            H_ret[i][j] = H_new[i][j]

    return W_ret,H_ret


In [7]:
def print_top_words_display(H, feature_names, n_top_words):
    table_output = [['Topic','topwords']]
    topics = []
    for topic_idx, topic in enumerate(H):
        top_index = ["Topic # %d" % topic_idx]
        top_words = [" ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])]
        values = [value for value in np.sort(topic)[:-n_top_words - 1:-1]]
        table_output.append(top_index + top_words)
        topics.append(" ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
    df_out = pd.DataFrame(table_output, index=None)
    display(HTML(tabulate(table_output, tablefmt='html')))
    return topics,df_out

In [8]:
def execute_tfidf(dataset,max_df=1,min_df=1,ngram=(0,2),stop_words=[]):
    tfidf_vectorizer = TfidfVectorizer(max_df=max_df, min_df=min_df,ngram_range=ngram,stop_words=stop_words)
    tfidf = tfidf_vectorizer.fit_transform(dataset)
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
    return tfidf,tfidf_feature_names

In [9]:
def execute_nmf(tfidf,n_topics = 10,n_components = 5):
    n_topics = n_topics
    n_components = n_components
    nmf = NMF(n_components=n_components,max_iter=400)
    W = nmf.fit_transform(tfidf)
    H = nmf.components_.transpose() 
    return nmf,W,H

In [10]:
def execute_stoc(W,H,n_final,n_components):
    n_words = H.shape[0]

    topXtop_norm = getIrredutibleMatrix(W,H,n_components)
    assign, t2 = joinTopics(n_components, topXtop_norm)


    tops = print_join(t2, H.transpose(), W, n_components, None, n_words)

    limiar = print_estats(t2)

    W_new, H_new = see_join(t2, W, H, limiar,n_final)
    return W_new,H_new

In [11]:
timeslice = []
corpus = ''
for ano in ['2013','2014','2015','2016','2017']:
    f = open('../../datasets/artigos/preprocessed/'+ano+'/artigosPre.txt','r')
    artigos = f.read()
    f.seek(0)
    timeslice.append(len(f.readlines()))
    corpus += artigos
open('new_corpus.txt','w').write(corpus)

640155

In [12]:
f = open('../acm_words.txt','r')
acm_words = f.read().replace('\n',' ').split()

In [56]:
fim = timeslice[0]
inicio = 0
for idx,ano in enumerate(['2013','2014','2015','2016','2017']):
    print(inicio,fim)
    vectorizer = CountVectorizer()
    time_corpus = corpus.split('\n')[inicio:fim]
    X = vectorizer.fit_transform(time_corpus)
    sum_words = X.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    words = [x[0] for x in words_freq]
    stop_words_without_acm_words = list(set(words) - set(acm_words))
    # TF-IDF
    max_df = 0.90
    min_df = 0.01
    ngram= (1,1)
    #NMF
    n_components = 10
    n_topics = 10
    # STOC
    #n_final = 30
    tfidf,tfidf_feature_names = execute_tfidf(time_corpus,max_df=max_df,min_df=min_df,ngram=ngram,stop_words = stop_words_without_acm_words)
    nmf,W,H = execute_nmf(n_components=n_components,n_topics=n_topics,tfidf=tfidf)
    topics_nmf,a = print_top_words_display(H.transpose(), tfidf_feature_names,n_topics)
    if not os.path.exists('data/'):
        os.mkdir('data/')
    np.savez('data/tfidf'+ano+'.npz',tfidf,tfidf_feature_names)
    np.savez('data/nmf'+ano+'.npz',W,H)
    inicio = fim
    if idx < len(['2013','2014','2015','2016','2017'])-1:
        fim = inicio + timeslice[idx+1]
    else:
        fim = inicio + timeslice[idx]


0 2133


Topic,topwords
Topic # 0,base cluster selection classification graph time detection framework recognition web
Topic # 1,network sensor wireless management application energy pattern structure architecture dynamic
Topic # 2,use machine classification web information power energy control source detection
Topic # 3,model framework management decision simulation application distribution interaction performance change
Topic # 4,analysis detection environment classification framework test source time recognition control
Topic # 5,system development control information detection time support management performance evaluation
Topic # 6,algorithm optimization time search graph tree selection evaluation cluster decision
Topic # 7,software process development line product test research business support agent
Topic # 8,image classification detection segmentation feature pattern computer recognition scale set
Topic # 9,design application self framework support distribution architecture control set process


2133 4317


Topic,topwords
Topic # 0,model support cloud simulation cell control evaluation game test decision
Topic # 1,network sensor wireless control cloud social service distribution content power
Topic # 2,use detection sensor search simulation information solution technology generation assessment
Topic # 3,system time support management information power evaluation decision interaction framework
Topic # 4,base framework information test detection classification web service cluster generation
Topic # 5,analysis framework performance social power search web simulation object decision
Topic # 6,algorithm tree search performance multi selection optimization distribution power level
Topic # 7,software application process management development project energy environment service knowledge
Topic # 8,image graph classification evaluation space recognition application pattern selection feature
Topic # 9,design optimization game evaluation test pattern computer architecture energy environment


4317 6652


Topic,topwords
Topic # 0,use image classification process evaluation detection decision quality patient parameter
Topic # 1,model simulation process support design decision time parameter distribution cloud
Topic # 2,base image classification detection heuristic information ontology machine service search
Topic # 3,network sensor wireless time distribution control content design dynamic pattern
Topic # 4,analysis image performance power evaluation game identification time level assessment
Topic # 5,system design power control service performance evaluation distribution support architecture
Topic # 6,algorithm design cluster optimization tree multi evaluation parameter image function
Topic # 7,software development design project support process evaluation research literature ontology
Topic # 8,application energy time cloud power optimization performance communication design health
Topic # 9,management process information framework knowledge project support decision environment energy


6652 9044


Topic,topwords
Topic # 0,model flow simulation environment context evaluation structure feature support multi
Topic # 1,use image evaluation machine optimization process information classification simulation flow
Topic # 2,network sensor design dynamic support detection structure performance information control
Topic # 3,base optimization detection classification image pattern cluster recognition selection knowledge
Topic # 4,analysis process information simulation function performance cluster pattern digital classification
Topic # 5,system distribution information design control evaluation computer architecture energy health
Topic # 6,algorithm image optimization cluster tree machine design multi search classification
Topic # 7,time graph process detection recognition image activity tree sensor classification
Topic # 8,software development process line evaluation support service architecture management environment
Topic # 9,application cloud service design evaluation performance support sensor mobile information


9044 11270


Topic,topwords
Topic # 0,base optimization control framework classification agent feature cluster simulation selection
Topic # 1,analysis process health test decision computer performance research dynamic structure
Topic # 2,model estimation test location performance simulation distance energy graph framework
Topic # 3,use machine optimization classification generation information prediction estimation recognition feature
Topic # 4,system control support performance information power energy management detection agent
Topic # 5,network neural sensor wireless detection service management classification function recognition
Topic # 6,algorithm optimization tree search location machine heuristic evaluation graph game
Topic # 7,time process control distance solution service wireless sensor code change
Topic # 8,software design application evaluation process management performance cloud product architecture
Topic # 9,image detection classification process graph identification feature assessment function quality


In [57]:
vectorizer = CountVectorizer()
full_corpus = corpus.split('\n')
X = vectorizer.fit_transform(full_corpus)
sum_words = X.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
words = [x[0] for x in words_freq]
stop_words_without_acm_words = list(set(words) - set(acm_words))
# TF-IDF
max_df = 0.90
min_df = 0.01
ngram= (1,1)
#NMF
n_components = 10
n_topics = 10
# STOC
#n_final = 30
tfidf,tfidf_feature_names = execute_tfidf(full_corpus,max_df=max_df,min_df=min_df,ngram=ngram,stop_words = stop_words_without_acm_words)
nmf,W,H = execute_nmf(n_components=n_components,n_topics=n_topics,tfidf=tfidf)
topics_nmf,a = print_top_words_display(H.transpose(), tfidf_feature_names,n_topics)
if not os.path.exists('data/'):
    os.mkdir('data/')
np.savez('data/tfidf.npz',tfidf,tfidf_feature_names)
np.savez('data/nmf.npz',W,H)

Topic,topwords
Topic # 0,use machine classification information optimization evaluation detection prediction change simulation
Topic # 1,model simulation support time process evaluation cloud decision distribution environment
Topic # 2,network sensor wireless management time detection design energy control distribution
Topic # 3,base classification detection cluster framework optimization selection web feature recognition
Topic # 4,analysis performance framework classification environment simulation identification health pattern decision
Topic # 5,system time control management information support power evaluation distribution performance
Topic # 6,algorithm optimization tree cluster search multi design time machine evaluation
Topic # 7,software process development management line project support test service architecture
Topic # 8,image classification graph detection process time feature identification recognition pattern
Topic # 9,application design optimization cloud evaluation energy performance graph service environment
